In [10]:
import numpy as np
import pandas as pd
import os
from pymatgen import MPRester
import re

In the above cell all imports are handled.
<br>`from pymatgen import MPRester` is used as an API tool to download the formula and bandwidth of many catalyst

In [2]:
exclude = ['O', 'H', 'He', 'Ne', 'Ar', 'As', 'Se', 'Kr', 'Tc', 'Xe', 'Pm', 'Hf', 'Os', 'Ir', 'Hg', 'Tl', 'Ac', 'Pa', 'U', 'Np', 'Pu']
include = ['Ag', 'Al', 'Au', 'B', 'Ba', 'Be', 'Bi', 'Br', 'C', 'Ca', 'Cd', 'Ce', 'Cl', 'Co', 'Cr', 'Cs', 'Cu', 'Dy', 'Er', 'Eu', 'F', 'Fe', 'Ga', 'Gd', 'Ge', 'Ho', 'I', 'In', 'K', 'La', 'Li', 'Lu', 'Mg', 'Mn', 'Mo', 'N', 'Na', 'Nb', 'Nd', 'Ni', 'P', 'Pb', 'Pd', 'Pr', 'Pt', 'Rb', 'Re', 'Rh', 'Ru', 'S', 'Sb', 'Sc', 'Si', 'Sm', 'Sn', 'Sr', 'Ta', 'Tb', 'Te', 'Th', 'Ti', 'Tm', 'V', 'W', 'Y', 'Yb', 'Zn', 'Zr']

`exclude`: Catalysts with these elements will be excludet in the dataset.
<br>`include`: Catalysts with these elements will be inclduet in the dataset.

In [3]:
m = MPRester('ifLyENSM2ZCJUWrl') 
data = m.query(criteria={"elements": {"$nin": exclude}, "nelements": {"$lt": 9}, "band_gap": {"$gt": 1e-4}}, properties=["pretty_formula", "band_gap"])

`m = MPRester('ifLyENSM2ZCJUWrl') ` builds a connection to the online database and 
<br>`data = m.query(...)` downloads the formula and band_gap of the catalysts

In [59]:
def composition_from_formula(formula):
    name = formula["pretty_formula"]
    band_gap = formula["band_gap"]
    elements_and_numbers = re.findall('[A-Z][^A-Z]*', name)
    columns = [re.sub(r'[0-9]+', '', s) for s in elements_and_numbers]
    columns.append('band_gap')
    stoechiometric_faktors = [1  if not any(char.isdigit() for char in element) else int(re.findall(r'\d+', element)[0]) for element in elements_and_numbers]
    content = np.array(stoechiometric_faktors) / max(stoechiometric_faktors)
    composition = {'line': name, 'content': np.append(content, band_gap), 'columns' : columns}
    return composition   


{'pretty_formula': 'Ba3PN', 'band_gap': 0.025499999999999856}
{'line': 'Ba3PN', 'content': array([1.        , 0.33333333, 0.33333333, 0.0255    ]), 'columns': ['Ba', 'P', 'N', 'band_gap']}


In [41]:
s = 'KMnO21'

a = re.findall(r'\d+', s)
print(a)

['21']
